In [64]:

from bs4 import BeautifulSoup
import sqlite3
import requests
import pandas as pd
from random import randint
import time
from time import sleep
from typing import List
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score

In [65]:
def sleepms(milliseconds: int):
    seconds = 0.001 * milliseconds
    sleep(seconds)
    
def random_wait():
    start = time.time()
    sleepms(randint(1, 250))

def get_fake_user_agent():
    return {"User-Agent":"PostmanRuntime/7.29.0"}


def getRetHeaders():
    url='https://www.ret.co.il/m/91935/specs'
    page = requests.get(url, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    product=soup.find('div',class_='single-product-tab')
    data=product.findAll('ul','list-group list-group-horizontal')
    myheaders=[]
    for alldata in data:
        myheaders.append(alldata.find('span',class_='text-dark font-weight-bold').text.strip())
    return myheaders



def getUrl(url):
    page = requests.get(url, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    MainClass=soup.find('ul', class_='nav nav-pills mb-2 justify-content-around justify-content-md-start col-xxl-10 offset-xxl-1 col-md-12 offset-md-0')
    ref=[]
    data=MainClass.find('a')
    technicalDataPage='https://www.ret.co.il'+data.get('href')
    return technicalDataPage

def getRetData(url):
    technicalDataPage=getUrl(url)
    page = requests.get(technicalDataPage, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    product=soup.find('div',class_='single-product-tab')
    dataLink=product.findAll('ul',class_='list-group list-group-horizontal')
    mydata=[]
    myheaders=[]
    for alldata in dataLink:
        try:
            mydata.append(alldata.find('li',class_='list-group-item col-8 rounded-0 text-600').text.strip())
        except:
            mydata.append('None')
    return mydata



In [66]:
def getRetToDataFrame(phonesURL):
    hrefLinks=[]
    allRetData=[]
    page = requests.get(phonesURL, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    MainClass=soup.findAll('a')
    for href in MainClass:
        if('/m/' in href.get('href')):
            hrefLinks.append('https://www.ret.co.il' + href.get('href'))
    for hrefLink in hrefLinks:
        allRetData.append(getRetData(hrefLink))
    return allRetData


In [67]:
def mainFunc(page_num_ret):
    head=getRetHeaders()
    url=f"https://www.ret.co.il/shop/cellphone?pageindex={page_num_ret}"
    data=getRetToDataFrame(url)
    df=pd.DataFrame(data,columns=head)
    return df

In [68]:
df_ret=mainFunc(1)
for i in range(2,10):
    df_ret=df_ret.append(mainFunc(i))
    random_wait()


C:\Users\david\AppData\Local\Temp\ipykernel_16752\2162949813.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\david\AppData\Local\Temp\ipykernel_16752\2162949813.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\david\AppData\Local\Temp\ipykernel_16752\2162949813.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\david\AppData\Local\Temp\ipykernel_16752\2162949813.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\david\AppData\Local\Temp\ipykernel_16752\216294

In [69]:
df_ret

,יצרן,מערכת הפעלה,שנת הכרזה,מבנה המכשיר,סדרה,Smartphone,רמת קרינה (SAR),ממשק הפעלה,צבעים,טלפון כשר,...,חיבור 3.5,רמקול חיצוני,רדיו FM,גובה,רוחב,עובי,משקל,קיבולת הסוללה,סוללה נשלפת,טעינה אלחוטית
0,Apple,iOS,2020,פאבלט,iPhone 12,סמארטפון,None,מסך מגע,None,None,...,ללא,זוג רמקולים,ללא רדיו,"146.1 מ""מ","71.5 מ""מ","7.4 מ""מ",164 גרם,None,לא,כולל
1,Apple,iOS,2019,פאבלט,iPhone 11,סמארטפון,None,מסך מגע,None,None,...,ללא,זוג רמקולים,ללא רדיו,"150.9 מ""מ","75.7 מ""מ","8.3 מ""מ",194 גרם,"3,110 mAh",לא,כולל
2,Samsung,None,2021,פאבלט,Galaxy S21,סמארטפון,0.71 W/kg,מסך מגע,None,None,...,ללא,זוג רמקולים,ללא רדיו,"165.1 מ""מ","75.6 מ""מ","8.9 מ""מ",227 גרם,"5,000 mAh",לא,כולל
3,Samsung,Android,2021,פאבלט,Galaxy S21,סמארטפון,0.46 W/kg,מסך מגע,None,None,...,ללא,זוג רמקולים,ללא רדיו,"151.7 מ""מ","71.2 מ""מ","7.9 מ""מ",169 גרם,"4,000 mAh",לא,כולל
4,Apple,iOS,2020,פאבלט,iPhone 12 Pro Max,סמארטפון,None,מסך מגע,None,None,...,ללא,זוג רמקולים,ללא רדיו,"160.8 מ""מ","78.1 מ""מ","7.4 מ""מ",228 גרם,None,לא,כולל
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,Samsung,Android,None,פאבלט,None,סמארטפון,None,מסך מגע,None,None,...,ללא,יחיד,None,"163.7 מ""מ","76.1 מ""מ","7.6 מ""מ",181 גרם,"5,000 mAh",לא,None
18,Samsung,Android,2021,פאבלט,Galaxy A52,סמארטפון,1.05 W/kg,מסך מגע,None,None,...,כולל,זוג רמקולים,כולל רדיו,"159.9 מ""מ","75.1 מ""מ","8.4 מ""מ",189 גרם,"4,500 mAh",לא,None
19,Samsung,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
20,Xiaomi,Android,None,פאבלט,None,סמארטפון,None,מסך מגע,None,None,...,None,זוג רמקולים,None,"164.2 מ""מ","76.1 מ""מ","8.1 מ""מ",202 גרם,"5,000 mAh",לא,ללא


In [70]:
def get_headers():
    url="https://www.zap.co.il/compmodels.aspx?modelid=1142728"
    page = requests.get(url, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    All_headers=['מחיר מינימלי','מחיר מקסימלי']
    headers=soup.find_all('div',class_="detailsRow")
    for header in headers:
        All_headers.append(header.find('div',class_="detailsRowTitletxt").text.strip())
    return All_headers


In [71]:
def get_properites_from_link(url):
    All_data=[]
    All_headers=[]
    page = requests.get(url, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    headers=soup.find_all('div',class_="detailsRow")
    titles = ['מחיר מינימלי','מחיר מקסימלי']
    prices = soup.find('div','PricesTxt').findAll('span') 
    if len(prices) == 2:
        All_data = [ prices[1].text, prices[0].text ]
    elif len(prices) == 1: 
        All_data = [ prices[0].text, prices[0].text ]
    else:
        All_data = [ None,None ]
    for header in headers:
        All_headers.append(header.find('div',class_="detailsRowTitletxt").text.strip())
        All_data.append(header.find('div',class_="detailsRowTxt").text.strip())
    return All_data

In [72]:
def get_data_from_zap(phonesURL):
    hrefLinks=[]
    all_zap_Data=[]
    Main_Url="https://www.zap.co.il"
    page = requests.get(phonesURL, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    links=soup.find_all('div',class_="MoreInfo")
    for link in links:
        hrefLinks.append(Main_Url+(link.find('a')['href']))
    for link in hrefLinks:
        all_zap_Data.append(get_properites_from_link(link))
    return all_zap_Data
    

In [73]:
def get_details_from_current_page(page_num_zap):
    url="https://www.zap.co.il/models.aspx?sog=e-cellphone&pageinfo={page_num_zap}"
    data=get_data_from_zap(url)
    df=pd.DataFrame(data,columns=get_headers())
    return df


In [74]:
df_zap=get_details_from_current_page(1)
for i in range(2,21):
    df_zap=df_zap.append(get_details_from_current_page(i))
    random_wait()
df_zap

C:\Users\david\AppData\Local\Temp\ipykernel_16752\2924271895.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_details_from_current_page(i))
C:\Users\david\AppData\Local\Temp\ipykernel_16752\2924271895.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_details_from_current_page(i))
C:\Users\david\AppData\Local\Temp\ipykernel_16752\2924271895.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_details_from_current_page(i))
C:\Users\david\AppData\Local\Temp\ipykernel_16752\2924271895.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_de

,מחיר מינימלי,מחיר מקסימלי,יצרן,מערכת הפעלה,שנת הכרזה,מבנה המכשיר,סדרה,Smartphone,רמת קרינה (SAR),ממשק הפעלה,...,חיבור 3.5,רמקול חיצוני,רדיו FM,גובה,רוחב,עובי,משקל,קיבולת הסוללה,סוללה נשלפת,טעינה אלחוטית
0,"3,465","4,649",Samsung,Android,2022,פאבלט,Galaxy S22,סמארטפון,1.05 W/kg,מסך מגע,...,ללא,זוג רמקולים,יעודכן בקרוב,"163.3 מ""מ","77.9 מ""מ","8.9 מ""מ",228 גרם,"5,000 mAh",לא,כולל
1,"1,514","1,949",Samsung,Android,2022,פאבלט,Galaxy A73,סמארטפון,1.28 W/kg,מסך מגע,...,יעודכן בקרוב,יחיד,יעודכן בקרוב,"163.7 מ""מ","76.1 מ""מ","7.6 מ""מ",181 גרם,"5,000 mAh",לא,ללא
2,"5,799","6,699",Apple,iOS,2022,פאבלט,iPhone 14 Pro Max,סמארטפון,לא זמין,מסך מגע,...,ללא,זוג רמקולים,ללא רדיו,"160.7 מ""מ","77.6 מ""מ","7.9 מ""מ",240 גרם,לא זמין,לא,כולל
3,"2,781","3,398",Apple,iOS,2021,פאבלט,iPhone 13,סמארטפון,יעודכן בקרוב,מסך מגע,...,ללא,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.7 מ""מ",174 גרם,לא זמין,לא,כולל
4,"3,197","3,965",Apple,iOS,2022,פאבלט,iPhone 14,סמארטפון,לא זמין,מסך מגע,...,ללא,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.8 מ""מ",172 גרם,לא זמין,לא,כולל
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,"3,870","4,799",Apple,iOS,2021,פאבלט,iPhone 13,סמארטפון,יעודכן בקרוב,מסך מגע,...,ללא,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.7 מ""מ",204 גרם,לא זמין,לא,כולל
20,"2,397","2,874",Xiaomi,Android,2022,פאבלט,12T Pro,סמארטפון,לא זמין,מסך מגע,...,ללא,זוג רמקולים,ללא רדיו,"163.1 מ""מ","75.9 מ""מ","8.6 מ""מ",205 גרם,"5,000 mAh",לא,לא זמין
21,"1,988","2,690",Apple,iOS,2019,פאבלט,iPhone 11,סמארטפון,לא זמין,מסך מגע,...,ללא,זוג רמקולים,ללא רדיו,"150.9 מ""מ","75.7 מ""מ","8.3 מ""מ",194 גרם,"3,110 mAh",לא,כולל
22,"1,289","1,949",Google,Android,2022,פאבלט,Pixel 6a,סמארטפון,לא זמין,מסך מגע,...,ללא,זוג רמקולים,ללא רדיו,"152.2 מ""מ","71.8 מ""מ","8.9 מ""מ",178 גרם,"4,410 mAh",לא,ללא


In [75]:
df=df_zap.append(df_ret)
df.to_csv('PhonesDF.csv',header=False)
df


C:\Users\david\AppData\Local\Temp\ipykernel_16752\2530705238.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df_zap.append(df_ret)


,מחיר מינימלי,מחיר מקסימלי,יצרן,מערכת הפעלה,שנת הכרזה,מבנה המכשיר,סדרה,Smartphone,רמת קרינה (SAR),ממשק הפעלה,...,רמקול חיצוני,רדיו FM,גובה,רוחב,עובי,משקל,קיבולת הסוללה,סוללה נשלפת,טעינה אלחוטית,Dual-SIM
0,"3,465","4,649",Samsung,Android,2022,פאבלט,Galaxy S22,סמארטפון,1.05 W/kg,מסך מגע,...,זוג רמקולים,יעודכן בקרוב,"163.3 מ""מ","77.9 מ""מ","8.9 מ""מ",228 גרם,"5,000 mAh",לא,כולל,NaN
1,"1,514","1,949",Samsung,Android,2022,פאבלט,Galaxy A73,סמארטפון,1.28 W/kg,מסך מגע,...,יחיד,יעודכן בקרוב,"163.7 מ""מ","76.1 מ""מ","7.6 מ""מ",181 גרם,"5,000 mAh",לא,ללא,NaN
2,"5,799","6,699",Apple,iOS,2022,פאבלט,iPhone 14 Pro Max,סמארטפון,לא זמין,מסך מגע,...,זוג רמקולים,ללא רדיו,"160.7 מ""מ","77.6 מ""מ","7.9 מ""מ",240 גרם,לא זמין,לא,כולל,NaN
3,"2,781","3,398",Apple,iOS,2021,פאבלט,iPhone 13,סמארטפון,יעודכן בקרוב,מסך מגע,...,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.7 מ""מ",174 גרם,לא זמין,לא,כולל,NaN
4,"3,197","3,965",Apple,iOS,2022,פאבלט,iPhone 14,סמארטפון,לא זמין,מסך מגע,...,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.8 מ""מ",172 גרם,לא זמין,לא,כולל,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,NaN,NaN,Samsung,Android,None,פאבלט,None,סמארטפון,None,מסך מגע,...,יחיד,None,"163.7 מ""מ","76.1 מ""מ","7.6 מ""מ",181 גרם,"5,000 mAh",לא,None,None
18,NaN,NaN,Samsung,Android,2021,פאבלט,Galaxy A52,סמארטפון,1.05 W/kg,מסך מגע,...,זוג רמקולים,כולל רדיו,"159.9 מ""מ","75.1 מ""מ","8.4 מ""מ",189 גרם,"4,500 mAh",לא,None,None
19,NaN,NaN,Samsung,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
20,NaN,NaN,Xiaomi,Android,None,פאבלט,None,סמארטפון,None,מסך מגע,...,זוג רמקולים,None,"164.2 מ""מ","76.1 מ""מ","8.1 מ""מ",202 גרם,"5,000 mAh",לא,ללא,כולל
